In [1]:
import pandas as pd

In [8]:
song = pd.read_csv()

In [9]:
song= song.dropna(how='all', axis=1)
song = song.drop(['End Position In Milliseconds','Event ID','Event Reason Hint Type','Feature Name','Media Duration In Milliseconds','Milliseconds Since Play','Personalized Name','Play Duration Milliseconds','Start Position In Milliseconds','Use Listening History'],axis=1)

In [10]:
spac = song[song['Container Type']=='ALBUM'].index
song.drop(spac, inplace = True)

In [18]:
spac = song[song['Container Type']=='RADIO'].index
song.drop(spac, inplace = True)

In [19]:
song

,Album Name,Auto Play,Container Type,,Event Received Timestamp,Event Start Timestamp,Event Type,Repeat Play,Shuffle Play,Song Name
0,NaN,AUTO_ON_CONTENT_UNSUPPORTED,PLAYLIST,NO_DISPLAY_TYPE_SPECIFIED,2023-01-01 10:38:00,2023-01-01 10:38,PLAY_START,REPEAT_ONE,SHUFFLE_OFF,灰色と青 ( + 菅田将暉 )
1,NaN,AUTO_ON_CONTENT_UNSUPPORTED,PLAYLIST,NO_DISPLAY_TYPE_SPECIFIED,2023-01-01 10:43:00,2023-01-01 10:43,PLAY_START,REPEAT_ONE,SHUFFLE_OFF,灰色と青 ( + 菅田将暉 )
2,NaN,AUTO_ON_CONTENT_UNSUPPORTED,PLAYLIST,NO_DISPLAY_TYPE_SPECIFIED,2023-01-01 10:44:00,2023-01-01 10:44,PLAY_START,REPEAT_OFF,SHUFFLE_OFF,バトンロード
3,NaN,AUTO_ON_CONTENT_UNSUPPORTED,PLAYLIST,NO_DISPLAY_TYPE_SPECIFIED,2023-01-01 10:48:00,2023-01-01 10:48,PLAY_START,REPEAT_OFF,SHUFFLE_OFF,LIAR
4,NaN,AUTO_ON_CONTENT_UNSUPPORTED,PLAYLIST,NO_DISPLAY_TYPE_SPECIFIED,2023-01-01 10:52:00,2023-01-01 10:52,PLAY_START,REPEAT_OFF,SHUFFLE_OFF,Wonderland
...,...,...,...,...,...,...,...,...,...,...
12678,NaN,AUTO_OFF,PLAYLIST,NO_DISPLAY_TYPE_SPECIFIED,2023-12-12 14:03:00,2023-12-12 14:03,PLAY_START,REPEAT_OFF,SHUFFLE_OFF,So Far Gone
12679,NaN,AUTO_OFF,PLAYLIST,NO_DISPLAY_TYPE_SPECIFIED,2023-12-12 14:06:00,2023-12-12 14:06,PLAY_START,REPEAT_OFF,SHUFFLE_OFF,Broken Heart of Gold (Japanese Version)
12680,NaN,AUTO_OFF,PLAYLIST,NO_DISPLAY_TYPE_SPECIFIED,2023-12-12 14:10:00,2023-12-12 14:10,PLAY_START,REPEAT_OFF,SHUFFLE_OFF,Nobody's Home
12681,NaN,AUTO_ON_CONTENT_UNSUPPORTED,PLAYLIST,NO_DISPLAY_TYPE_SPECIFIED,2023-12-12 22:15:00,2023-12-12 22:15,PLAY_START,REPEAT_OFF,SHUFFLE_OFF,Liar


In [31]:
from datetime import datetime, timedelta
from collections import defaultdict

RECENT_THRESHOLD = 60  # 최근 몇 일 동안 들은 노래에 가중치를 부여할 것인지
SELECTION_WEIGHT = 1.7  # 직접 선택한 노래에 부여할 가중치


song['Event Received Timestamp'] = pd.to_datetime(song['Event Received Timestamp'])
song_dates = defaultdict(lambda: {'dates': [], 'score': 1.0})
top_songs = song['Song Name'].value_counts().nlargest(15).index

desired_date = datetime(2023, 12, 12)

for i, row in song.iterrows():
    song_name = row['Song Name']
    score = 2.0 if (desired_date - row['Event Received Timestamp']).days <= RECENT_THRESHOLD else 1.0
    
    if row['Auto Play'] == 'AUTO_OFF':
        score *= SELECTION_WEIGHT
    if row['Repeat Play'] == 'REPEAT_ONE':
        score *= 1.6
    if song_name in top_songs:
        score *= 1.8
        
    song_dates[song_name]['dates'].append(row['Event Received Timestamp'])
    song_dates[song_name]['score'] = score


top_songs = sorted(song_dates.items(), key=lambda x: x[1]['score'], reverse=True)[:15]
 
for rank, (song_name, data_info) in enumerate(top_songs, start=1):
    print(f"Ranking: {rank}")
    print(f"Song: {song_name}")
    print(f"Score: {data_info['score']}")
    print("\n")    

Ranking: 1
Song: ノールス
Score: 5.5


Ranking: 2
Song: Sukidakara
Score: 5.5


Ranking: 3
Song: 過去に囚われている
Score: 5.5


Ranking: 4
Song: Two Creatures
Score: 5.5


Ranking: 5
Song: Kid
Score: 5.5


Ranking: 6
Song: アイドル
Score: 5.5


Ranking: 7
Song: Internet Overdose
Score: 5.5


Ranking: 8
Song: Generation
Score: 5.5


Ranking: 9
Song: nan
Score: 4.300000000000001


Ranking: 10
Song: 泥の分際で私だけの大切を奪おうだなんて
Score: 3.8


Ranking: 11
Song: Easy On Me (Apple Music Home Session)
Score: 3.8


Ranking: 12
Song: まるごと
Score: 3.8


Ranking: 13
Song: Wherever You Are
Score: 3.8


Ranking: 14
Song: ブルーベリー・ナイツ
Score: 3.8


Ranking: 15
Song: Marunouchi Sadistic
Score: 3.8


